#### Question 3: Improvement 2

In [53]:
import pandas as pd
import numpy as np 
import plotly.graph_objects as go
from datetime import date

**Data**

In [92]:
# Financial Ratios
sp500_monthly_ratios = pd.read_csv('data/SP500_Monthly.csv')
sp600_monthly_ratios = pd.read_csv('data/SP600_Monthly.csv')

# Prices
sp500_daily_prices = pd.read_csv('data/SP500_daily.csv') 
sp600_daily_prices = pd.read_csv('data/SP600_daily.csv') 

# Table
sp500_table = pd.read_csv('data/sp500_table.csv')
sp600_table = pd.read_csv('data/sp600_table.csv')

/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_59766/731905049.py:6: DtypeWarning:

Columns (6,9,19,22,24,49,57) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_59766/731905049.py:7: DtypeWarning:

Columns (5,6,9,19,22,24,49,57) have mixed types. Specify dtype option on import or set low_memory=False.



In [93]:
from collections import defaultdict
sp500_table = sp500_table.drop_duplicates(subset=['GICS Sector','Symbol'])[['GICS Sector','Symbol']]
sp500_sectors = defaultdict(list)

for sector in sp500_table['GICS Sector'].unique():
    for symbol in sp500_table[sp500_table['GICS Sector'] == sector]['Symbol'].unique():
        sp500_sectors[sector].append(symbol)

sp600_table = sp600_table.drop_duplicates(subset=['GICS Sector','Symbol'])[['GICS Sector','Symbol']]
sp600_sectors = defaultdict(list)

for sector in sp600_table['GICS Sector'].unique():
    for symbol in sp600_table[sp600_table['GICS Sector'] == sector]['Symbol'].unique():
        sp600_sectors[sector].append(symbol)

In [94]:
sp600_prices = (sp600_daily_prices[['date','TICKER','OPENPRC']]
 .groupby(["date", "TICKER"])["OPENPRC"]
 .last()
 .reset_index()
 .pivot(index="date", columns="TICKER", values="OPENPRC")
 .dropna(axis = 1)
)

sp600_cfac = (sp600_daily_prices[['date','TICKER','CFACPR']]
 .groupby(["date", "TICKER"])["CFACPR"]
 .last()
 .reset_index()
 .pivot(index="date", columns="TICKER", values="CFACPR")
 .dropna(axis = 1)
)

sp600_prices = (sp600_prices / sp600_cfac).dropna(axis = 1)

sp600_returns = sp600_prices.pct_change().dropna()
sp600_returns.index = pd.to_datetime(sp600_returns.index).date

sp500_prices = (sp500_daily_prices[['date','TICKER','OPENPRC']]
 .groupby(["date", "TICKER"])["OPENPRC"]
 .last()
 .reset_index()
 .pivot(index="date", columns="TICKER", values="OPENPRC")
 .dropna(axis = 1)
)

sp500_cfac = (sp500_daily_prices[['date','TICKER','CFACPR']]
 .groupby(["date", "TICKER"])["CFACPR"]
 .last()
 .reset_index()
 .pivot(index="date", columns="TICKER", values="CFACPR")
 .dropna(axis = 1)
)

sp500_prices = (sp500_prices / sp500_cfac).dropna(axis = 1)

sp500_returns = sp500_prices.pct_change().dropna()
sp500_returns.index = pd.to_datetime(sp500_returns.index).date

sp500_monthly_ratios = (sp500_monthly_ratios
 .set_index(['public_date'])[['TICKER','roe','roa','ptb']]
 )
sp500_monthly_ratios.index = pd.to_datetime(sp500_monthly_ratios.index).date

sp600_monthly_ratios = (sp600_monthly_ratios
 .set_index(['public_date'])[['TICKER','roe','roa','ptb']]
 )
sp600_monthly_ratios.index = pd.to_datetime(sp600_monthly_ratios.index).date

**Backtest**

In [96]:
dates = pd.to_datetime(sp500_monthly_ratios.index.unique()).date
dates = sorted(dates)

In [101]:
small_cap_tickers = set(sp600_monthly_ratios['TICKER']) & set(sp600_prices.columns)
small_cap_weights = pd.Series(0.0, index= small_cap_tickers, dtype=float)
large_cap_tickers = set(sp500_monthly_ratios['TICKER']) & set(sp500_prices.columns)
large_cap_weights = pd.Series(0.0, index= large_cap_tickers, dtype=float)

portfolio_returns = pd.Series(0.0,index = sp500_returns.index,dtype=float)
long_portfolio_returns = pd.Series(0.0,index = sp500_returns.index,dtype=float)
short_portfolio_returns = pd.Series(0.0,index = sp500_returns.index,dtype=float)

# Filter Price Tickers
# sp500_prices = sp500_prices[list(large_cap_tickers)]
# sp600_prices = sp600_prices[list(small_cap_tickers)]

# Filter Ratios
sp500_monthly_ratios = sp500_monthly_ratios[sp500_monthly_ratios['TICKER'].isin(list(large_cap_tickers))]
sp600_monthly_ratios = sp600_monthly_ratios[sp600_monthly_ratios['TICKER'].isin(list(small_cap_tickers))]

rebalance = 30
portfolio_start_date = None  # Track when portfolio is first constructed

for i in range(len(sp500_returns.index)):

    dt = sp500_returns.index[i]
    # Skip return calculations until after the first rebalance
    if portfolio_start_date is not None and dt > portfolio_start_date:
        small_cap_returns = (small_cap_weights * sp600_returns.loc[dt]).sum()
        large_cap_returns = (large_cap_weights * sp500_returns.loc[dt]).sum()

        long_portfolio_returns[dt] = small_cap_returns
        short_portfolio_returns[dt] = large_cap_returns
        gross_exposure = small_cap_weights.sum() + abs(large_cap_weights).sum()  # Should be 2.0
        portfolio_returns[dt] = (small_cap_returns + large_cap_returns) / gross_exposure

    # Rebalance Portfolio
    if dt in dates:

        # Set portfolio start date
        portfolio_start_date = dt  

        # Reset Weights
        large_cap_weights = pd.Series(0.0, index=large_cap_tickers, dtype=float)
        small_cap_weights = pd.Series(0.0, index=small_cap_tickers, dtype=float)

        for sector in sp600_sectors:
                
            # Small Cap Selection
            filtered_small = sp600_monthly_ratios[sp600_monthly_ratios['TICKER'].isin(sp600_sectors[sector])]
            filtered_small = filtered_small.loc[filtered_small.index == dt]
            long_tickers = filtered_small.sort_values(by=['roe', 'roa', 'ptb'], ascending=[False, False, False])['TICKER'].iloc[:5].values

            small_cap_weights[long_tickers] = 1 / len(long_tickers) if len(long_tickers) != 0 else 0

            # Large Cap Selection
            filtered_large = sp500_monthly_ratios[sp500_monthly_ratios['TICKER'].isin(sp500_sectors[sector])]
            filtered_large = filtered_large.loc[filtered_large.index == dt]
            short_tickers = filtered_large.sort_values(by=['roe', 'roa', 'ptb'], ascending=[True, True, True])['TICKER'].iloc[:5].values

            large_cap_weights[short_tickers] = -1 / len(short_tickers) if len(short_tickers) != 0 else 0

            if filtered_large.index[0] != dt or filtered_small.index[0] != dt:
                break
        # Normalize Weights
        small_cap_weights /= (small_cap_weights.sum())
        large_cap_weights /= (abs(large_cap_weights).sum())

        if small_cap_weights.sum() != 1 or large_cap_weights.sum() != -1:
            print(dt)

2013-07-31
2013-10-31
2014-01-31
2014-02-28
2014-03-31
2014-04-30
2020-03-31
2020-04-30
2023-11-30


In [105]:
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x = portfolio_returns.index,
        y = portfolio_returns.cumsum()
    )
)

fig.update_layout(title = 'Ratios Strategy')
fig.update_yaxes(title = 'Cumulative Returns')
fig.show()

In [133]:
filtered_small

,TICKER,roe,roa,ptb
2023-11-30,TDS,-0.003,0.072,0.369
2023-11-30,SHEN,-0.004,0.072,1.506
2023-11-30,CCOI,-0.047,0.101,3.739
2023-11-30,CNK,0.536,0.119,4.616
2023-11-30,TTGT,0.058,0.046,3.795
2023-11-30,SATS,0.096,0.300,0.136


In [134]:
filtered_large

,TICKER,roe,roa,ptb
2023-11-30,NWSA,0.016,0.081,1.044
2023-11-30,DIS,0.022,0.070,1.592
2023-11-30,OMC,0.393,0.096,4.245
2023-11-30,IPG,0.257,0.097,3.199
2023-11-30,VZ,0.155,0.125,1.133
2023-11-30,T,-0.065,0.105,0.730
2023-11-30,TTWO,-0.153,0.038,3.089
2023-11-30,NFLX,0.211,0.130,9.384
2023-11-30,CMCSA,0.137,0.148,1.506
2023-11-30,LYV,NaN,0.087,53.202
